# Working with Local Dataset

In this tutorial, we will show how to use your own local dataset with the Dataset class. The Dataset class can help you to manage and process your eyetracking data.

## Preparations

We import `pymovements` as the alias `pm` for convenience.

In [ ]:
import pymovements as pm

For demonstration purposes, we will use the raw data provided by the Toy dataset, a sample dataset that comes with *pymovements*.

We will download the resources of this dataset the directory to simulate a local dataset for you.
All downloaded archive files are automatically extracted and then removed.
The directory of the dataset will be `data/my_dataset`.

After that we won't use the python class anymore and delete the object
(the files on your system will stay in place).
Don't worry if you're confused about these lines as they are not relevant to your use case.

Just keep in mind that we now have some files with gaze data in the directory `data/my_dataset`.

In [ ]:
toy_dataset = pm.Dataset('ToyDataset', path='data/my_dataset')
toy_dataset.download(remove_finished=True)

del toy_dataset

## Define your Experiment

To use the Dataset class, we first need to create an Experiment instance. This class represents the properties of the experiment, such as the screen dimensions and sampling rate.

In [ ]:
experiment = pm.gaze.Experiment(
    screen_width_px=1280,
    screen_height_px=1024,
    screen_width_cm=38,
    screen_height_cm=30.2,
    distance_cm=68,
    origin='lower left',
    sampling_rate=1000,
)

## Parameters for File Parsing

We also define a `filename_format` which is a pattern expression used to match and extract values from filenames of data files in the dataset. For example, `r'trial_{text_id:d}_{page_id:d}.csv'` will match filenames that follow the pattern `trial_{text_id}_{page_id}.csv` and extract the values of `text_id` and `page_id` for each file.

In [ ]:
filename_format = r'trial_{text_id:d}_{page_id:d}.csv'

Both values of `text_id` and `page_id` are numeric. We can use a map to define the casting of these values.

In [ ]:
filename_format_dtypes = {
    'text_id': int,
    'page_id': int,
}

We can also adjust how the CSV files are read.
Here, we specify that the separator in the CSV files is a tab (**'\t'**).

In [ ]:
custom_read_kwargs = {
    'separator': '\t',
}

## Column Definitions

The `trial_columns` argument can be used to specify which columns define a single trial.

This is important for correctly applying all preprocessing methods.

For this very small single user dataset a trial ist just defined by `text_id` and `page_id`.

In [ ]:
trial_columns = ['text_id', 'page_id']

The `time_column` and `pixel_columns` arguments can be used to correctly map the columns in your dataframes. When specifying a `time_column` it is also necessary to specify the `time_unit` of the column.

Depending on the content of your dataset, you can alternatively also provide `position_columns`, `velocity_columns` and `acceleration_columns`.

Specifying these columns is needed for correctly applying preprocessing methods. For example, if you want to apply the `pix2deg` method, you will need to specify `pixel_columns` accordingly.

If your dataset has gaze positions available only in degrees of visual angle, you have to specify the `position_columns` instead.

In [ ]:
time_column = 'timestamp'
time_unit = 'ms'
pixel_columns = ['x', 'y']

## Define and load the Dataset

Next we use all these definitions and create a `DatasetDefinition` by passing in the root directory, Experiment instance, and other optional parameters such as the filename regular expression and custom CSV reading parameters.

In [ ]:
dataset_definition = pm.DatasetDefinition(
    name='my_dataset',
    experiment=experiment,
    filename_format=filename_format,
    filename_format_dtypes=filename_format_dtypes,
    custom_read_kwargs=custom_read_kwargs,
    time_column=time_column,
    time_unit=time_unit,
    pixel_columns=pixel_columns,
)

Finally we create a `Dataset` instance by using the `DatasetDefinition` and specifying the directory path.

In [ ]:
dataset = pm.Dataset(
    definition=dataset_definition,
    path='data/my_dataset/',
)

If we have a root data directory which holds all your local datasets we can further need to define the paths of the dataset.

The `dataset`, `raw`, `preprocessed`, and `events` parameters define the names of the directories for the dataset, raw data, preprocessed data, and events data, respectively.

In [ ]:
dataset_paths = pm.DatasetPaths(
    root='data/',
    raw='raw',
    preprocessed='preprocessed',
    events='events',
)

dataset = pm.Dataset(
    definition=dataset_definition,
    path=dataset_paths,
)

Now let's load the dataset into memory. Here we select a subset including the first page of texts with ID 1 and 2.

In [ ]:
subset = {
    'text_id': [1, 2],
    'page_id': 1,
}

dataset.load(subset=subset)

## Use the Dataset

Once we have created the Dataset instance, we can use its methods to preprocess and analyze data in our local dataset.

In [ ]:
dataset.gaze[0].frame

Here we use the `pix2deg` method to convert the pixel coordinates to degrees of visual angle.

In [ ]:
dataset.pix2deg()

dataset.gaze[0].frame

We can use the `pos2vel` method to calculate the velocity of the gaze position.

In [ ]:
dataset.pos2vel(method='savitzky_golay', degree=2, window_length=7)

dataset.gaze[0].frame